In [ ]:
%pip install torch torchvision
%pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from torch import nn
from torchvision.transforms import ToTensor
import glob
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import random
from PIL import Image
import os
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision import transforms

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/ADL Term 6/Project/dataset/train'
test_dir = '/content/drive/MyDrive/Colab Notebooks/ADL Term 6/Project/dataset/test'

train_dir, test_dir

('/content/drive/MyDrive/Colab Notebooks/ADL Term 6/Project/dataset/train',
 '/content/drive/MyDrive/Colab Notebooks/ADL Term 6/Project/dataset/test')

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize(size=(512, 512)),
    transforms.ToTensor() 
])

In [ ]:
train_data = datasets.ImageFolder(root=train_dir, 
                                  transform=data_transform,
                                  target_transform=None)

test_data = datasets.ImageFolder(root=test_dir, 
                                 transform=data_transform)

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

Train data:
Dataset ImageFolder
    Number of datapoints: 11424
    Root location: /content/drive/MyDrive/Colab Notebooks/ADL Term 6/Project/dataset/train
    StandardTransform
Transform: Compose(
               Resize(size=(512, 512), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 612
    Root location: /content/drive/MyDrive/Colab Notebooks/ADL Term 6/Project/dataset/test
    StandardTransform
Transform: Compose(
               Resize(size=(512, 512), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )


In [ ]:
#data_augmentation = transforms.Compose([
    #transforms.RandomRotation(degrees=22),
    #transforms.RandomHorizontalFlip(p=0.2),
    #transforms.RandomVerticalFlip(p=0.2),
    #transforms.ToTensor()  
#])

In [ ]:
#train_data = datasets.ImageFolder(root=train_dir,
                                  #transform=data_augmentation, 
                                  #target_transform=None) 

#test_data = datasets.ImageFolder(root=test_dir, 
                                 #transform=data_augmentation)

#print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 512, 512])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=3,              
                out_channels=96,            
                kernel_size=5,              
                stride=1,                   
                padding=1,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(96, 256, 11, 1, 1),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.conv3 = nn.Sequential(         
            nn.Conv2d(256, 384, 3, 1, 1),     
            nn.ReLU(),                                      
        )
        self.conv4 = nn.Sequential(         
            nn.Conv2d(384, 384, 3, 1, 1),     
            nn.ReLU(),                                   
        )
        self.conv5 = nn.Sequential(         
            nn.Conv2d(384, 256, 3, 1, 1),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.dropout = nn.Dropout(0.4)
        self.out1 = nn.Linear(43264, 1000)
        self.out2 = nn.Linear(1000, 1000)
        self.out3 = nn.Linear(1000, 2)
    def forward(self, x):
        x = self.conv1(x)     
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.out1(x)
        x = self.out2(x)       
        output = self.out3(x)
        return output

model = CNN().to(device)
print(model)

Using cuda device
CNN(
  (conv1): Sequential(
    (0): Conv2d(3, 96, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(96, 256, kernel_size=(11, 11), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(256, 384, kernel_size=(11, 11), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv4): Sequential(
    (0): Conv2d(384, 384, kernel_size=(11, 11), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv5): Sequential(
    (0): Conv2d(384, 256, kernel_size=(11, 11), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dropout): Dropout(p=0.4, inplace=False)
  (out1): Linear(in_features=614656, out_features=1000, bias=True)
  (out2): L

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


OutOfMemoryError: ignored